# week04

In [1]:
import random

import pandas as pd
import pymysql

## 数据库及预定义参数
这里需要把settings更换成实际值

In [2]:
MYSQL_SETTINGS = {
    'host': 'localhost',
    'port': 3306,
    'user': '*******',
    'password': '*******',
    'db': '*******',
    'charset': 'utf8'
}

SQL_CREATE_TABLE = """
CREATE TABLE IF NOT EXISTS `{}`(
   `id` INT UNSIGNED AUTO_INCREMENT,
   `name` VARCHAR(255) NOT NULL,
   `age` INTEGER NOT NULL,
   PRIMARY KEY (`id`)
)ENGINE=InnoDB DEFAULT CHARSET=utf8;
"""
SQL_INSERT = """
INSERT INTO `{}`
(`name`, `age`)
VALUES 
('{}', '{}');
"""

SQL_INIT_TABLE = "DROP TABLE IF EXISTS {}"

## 数据库初始化

In [3]:
db_conn = pymysql.connect(**MYSQL_SETTINGS)
db_cur = db_conn.cursor()
db_cur.execute(SQL_INIT_TABLE.format('data'))
db_cur.execute(SQL_INIT_TABLE.format('table1'))

db_cur.execute(SQL_CREATE_TABLE.format('data'))
db_cur.execute(SQL_CREATE_TABLE.format('table1'))

db_cur.close()
db_conn.commit()
db_conn.close()

## 随机新建数据库和DataFrame

In [4]:
db_conn = pymysql.connect(**MYSQL_SETTINGS)
db_cur = db_conn.cursor()
data_data = []
table1_data = []
for i in range(10):
    name = ''.join([chr(random.randint(97, 122)) for _ in range(3)])
    age = random.randint(20, 70)
    data_data.append([i + 1, name, age])
    db_cur.execute(SQL_INSERT.format('data', name, age))

for i in range(10):
    name = ''.join([chr(random.randint(97, 122)) for _ in range(3)])
    age = random.randint(20, 70)
    table1_data.append([i + 1, name, age])
    db_cur.execute(SQL_INSERT.format('table1', name, age))
db_cur.close()
db_conn.commit()
db_conn.close()

data = pd.DataFrame(data_data, columns=['id', 'name', 'age'])
table1 = pd.DataFrame(table1_data, columns=['id', 'name', 'age'])

1. SELECT * FROM data;

In [5]:
data

,id,name,age
0,1,vgy,50
1,2,zyk,44
2,3,gmy,60
3,4,hsh,43
4,5,fvm,64
5,6,hje,68
6,7,xgj,60
7,8,clz,24
8,9,oat,34
9,10,zfm,27


2. SELECT * FROM data LIMIT 10;

In [6]:
data.head(10)

,id,name,age
0,1,vgy,50
1,2,zyk,44
2,3,gmy,60
3,4,hsh,43
4,5,fvm,64
5,6,hje,68
6,7,xgj,60
7,8,clz,24
8,9,oat,34
9,10,zfm,27


3. SELECT id FROM data;  //id 是 data 表的特定一列

In [7]:
data.id

0     1
1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9    10
Name: id, dtype: int64

4. SELECT COUNT(id) FROM data;

In [8]:
data.id.count()

10

5. SELECT * FROM data WHERE id<1000 AND age>30;

In [9]:
data[(data.id < 1000) & (data.age > 30)]

,id,name,age
0,1,vgy,50
1,2,zyk,44
2,3,gmy,60
3,4,hsh,43
4,5,fvm,64
5,6,hje,68
6,7,xgj,60
8,9,oat,34


6. SELECT id,COUNT(DISTINCT order_id) FROM table1 GROUP BY id;

In [10]:
table1.groupby('id').age.nunique()

id
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
Name: age, dtype: int64

7. SELECT * FROM table1 t1 INNER JOIN table2 t2 ON t1.id = t2.id;

In [11]:
pd.merge(data, table1, on='id', how='inner')

,id,name_x,age_x,name_y,age_y
0,1,vgy,50,end,56
1,2,zyk,44,zeq,31
2,3,gmy,60,lxy,67
3,4,hsh,43,bvk,29
4,5,fvm,64,adl,41
5,6,hje,68,zbh,52
6,7,xgj,60,uui,59
7,8,clz,24,oho,26
8,9,oat,34,unu,68
9,10,zfm,27,kgm,33


8. SELECT * FROM table1 UNION SELECT * FROM table2;

In [12]:
pd.concat([data, table1]).drop_duplicates()

,id,name,age
0,1,vgy,50
1,2,zyk,44
2,3,gmy,60
3,4,hsh,43
4,5,fvm,64
5,6,hje,68
6,7,xgj,60
7,8,clz,24
8,9,oat,34
9,10,zfm,27


9. DELETE FROM table1 WHERE id=10;

In [13]:
# 方法1
table1[table1.id != 10]

,id,name,age
0,1,end,56
1,2,zeq,31
2,3,lxy,67
3,4,bvk,29
4,5,adl,41
5,6,zbh,52
6,7,uui,59
7,8,oho,26
8,9,unu,68


In [14]:
# 方法2
table1.drop(table1[table1.id == 10].index)

,id,name,age
0,1,end,56
1,2,zeq,31
2,3,lxy,67
3,4,bvk,29
4,5,adl,41
5,6,zbh,52
6,7,uui,59
7,8,oho,26
8,9,unu,68


10. ALTER TABLE table1 DROP COLUMN column_name;

In [15]:
# 方法1
table1.drop(['name'], axis=1)

,id,age
0,1,56
1,2,31
2,3,67
3,4,29
4,5,41
5,6,52
6,7,59
7,8,26
8,9,68
9,10,33


In [16]:
# 方法2
table1.drop(columns=['name'])

,id,age
0,1,56
1,2,31
2,3,67
3,4,29
4,5,41
5,6,52
6,7,59
7,8,26
8,9,68
9,10,33
